# Polyglot Notebooks → Verso Import

This notebook uses **Polyglot Notebooks** conventions for F# cells.
When imported into Verso, the `JupyterFSharpPostProcessor` automatically converts:

- `#!fsharp` / `#!f#` magic lines → sets cell language to F#
- `#!set` directives → `Variables.Set(...)` calls
- `#!share` directives → `Variables.Get(...)` calls
- Cell metadata `dotnet_interactive.language: fsharp` → F# language cells

Try importing this `.ipynb` file into Verso to see the conversion in action.

## Using `#!fsharp` Magic

The `#!fsharp` line tells Polyglot Notebooks to use the F# kernel.
Verso strips this line and sets the cell language automatically.

In [ ]:
#!fsharp
// Define a record type
type Measurement = {
    Location: string
    Temperature: float
    Humidity: float
    Timestamp: string
}

let readings = [
    { Location = "Lab A"; Temperature = 22.5; Humidity = 45.0; Timestamp = "2026-02-16 09:00" }
    { Location = "Lab B"; Temperature = 23.1; Humidity = 42.0; Timestamp = "2026-02-16 09:00" }
    { Location = "Lab C"; Temperature = 21.8; Humidity = 48.0; Timestamp = "2026-02-16 09:00" }
]

printfn "Loaded %d readings" readings.Length

## Using `#!f#` Alternate Syntax

The `#!f#` shorthand also works and is converted the same way.

In [ ]:
#!f#
// Compute averages using the readings from the previous cell
let avgTemp = readings |> List.averageBy (fun r -> r.Temperature)
let avgHumidity = readings |> List.averageBy (fun r -> r.Humidity)

printfn "Average temperature: %.1f°C" avgTemp
printfn "Average humidity: %.1f%%" avgHumidity

## Cell Metadata Detection

This cell uses `dotnet_interactive.language: fsharp` metadata instead of a magic line.
Verso detects this and sets the cell language accordingly.

In [ ]:
// This cell is detected as F# via metadata, no magic line needed
type Status = Normal | Warning | Critical

let classify temp =
    if temp > 25.0 then Critical
    elif temp > 23.0 then Warning
    else Normal

let statuses =
    readings
    |> List.map (fun r -> r.Location, classify r.Temperature)

for (loc, status) in statuses do
    printfn "%s: %A" loc status

## Variable Sharing with `#!set`

In Polyglot Notebooks, `#!set` passes values between kernels.
Verso converts `#!set --name <n> --value @fsharp:<expr>` into a `Variables.Set(...)` call.

In [ ]:
#!set --name threshold --value @fsharp:23.5

## Variable Sharing with `#!share`

The `#!share` directive imports a variable from another kernel.
Verso converts this into a `let` binding that reads from the variable store.

In [ ]:
#!share --from csharp avgTemp

## Mixed Magic and Code

A common pattern: `#!fsharp` magic followed by multiple lines of code.
On import, only the magic line is removed; all code is preserved.

In [ ]:
#!fsharp
// Summary report using pattern matching and string formatting
type Report = {
    Title: string
    AvgTemperature: float
    AvgHumidity: float
    ReadingCount: int
    Status: Status
}

let report = {
    Title = "Lab Environment Summary"
    AvgTemperature = avgTemp
    AvgHumidity = avgHumidity
    ReadingCount = readings.Length
    Status = classify avgTemp
}

printfn "=== %s ===" report.Title
printfn "Temperature: %.1f°C (%A)" report.AvgTemperature report.Status
printfn "Humidity:    %.1f%%" report.AvgHumidity
printfn "Readings:    %d" report.ReadingCount